# GridStatus Data Exploration

This notebook explores energy grid data using the GridStatus library.

## Setup
First time setup:
```bash
pip install gridstatus pandas matplotlib plotly seaborn jupyterlab
```

In [ ]:
import gridstatus
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import os

# Set up paths for data caching
DATA_DIR = '../data/raw'
os.makedirs(DATA_DIR, exist_ok=True)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
%matplotlib inline

## 1. Explore Available ISOs

GridStatus supports multiple Independent System Operators (ISOs).
CAISO (California) has the most solar generation data.

In [ ]:
# List all available ISOs
print("Available ISOs:")
for iso_name in dir(gridstatus):
    if iso_name.isupper() and not iso_name.startswith('_'):
        print(f"  - {iso_name}")

# Initialize CAISO (California - lots of solar)
caiso = gridstatus.CAISO()

## 2. Fetch Fuel Mix Data

Fuel mix shows generation by source (solar, wind, natural gas, etc.)

In [ ]:
# Define date range - start with a week of data
end_date = datetime.now()
start_date = end_date - timedelta(days=7)

# Format dates for API
start_str = start_date.strftime('%Y-%m-%d')
end_str = end_date.strftime('%Y-%m-%d')

print(f"Fetching data from {start_str} to {end_str}...")

# Check if we have cached data
cache_file = f'{DATA_DIR}/fuel_mix_{start_str}_{end_str}.csv'

if os.path.exists(cache_file):
    print(f"Loading from cache: {cache_file}")
    fuel_mix = pd.read_csv(cache_file, parse_dates=['Time'])
else:
    print("Fetching from API...")
    fuel_mix = caiso.get_fuel_mix(start=start_str, end=end_str)
    # Save to cache
    fuel_mix.to_csv(cache_file, index=False)
    print(f"Saved to cache: {cache_file}")

print(f"\nData shape: {fuel_mix.shape}")
fuel_mix.head(10)

## 3. Explore Data Structure

In [ ]:
# Check columns and data types
print("Column info:")
print(fuel_mix.info())

print("\nUnique fuel types:")
if 'Fuel' in fuel_mix.columns:
    print(fuel_mix['Fuel'].unique())

print("\nBasic statistics:")
fuel_mix.describe()

## 4. Quick Visualization: Solar Generation Over Time

In [ ]:
# Plot solar generation over the week
fig = px.line(
    fuel_mix, 
    x='Time', 
    y='Solar',
    title='Solar Generation Over Time (CAISO) - Past 7 Days',
    labels={'Solar': 'Megawatts', 'Time': 'Date/Time'}
)
fig.update_layout(
    hovermode='x unified',
    xaxis_title='Date/Time',
    yaxis_title='Solar Generation (MW)',
    height=500
)
fig.show()

# Print some interesting stats
print(f"\nSolar Generation Stats:")
print(f"  Peak generation: {fuel_mix['Solar'].max():,} MW")
print(f"  Average daytime generation: {fuel_mix[fuel_mix['Solar'] > 0]['Solar'].mean():.0f} MW")
print(f"  Peak time: {fuel_mix.loc[fuel_mix['Solar'].idxmax(), 'Time']}")

## 5. Compare Two Days: Sunny vs Cloudy

TODO: Pick specific dates based on patterns observed above

In [ ]:
# Example: Compare two specific days
# day1 = '2024-06-15'  # Sunny day
# day2 = '2024-06-20'  # Cloudy day

# Filter data for these days and create comparison plot
# (Implementation depends on data structure discovered above)

## 6. Explore Other Data Types

In [ ]:
# Get load (demand) data
# load_data = caiso.get_load(start=start_str, end=end_str)

# Get pricing data (LMP - Locational Marginal Price)
# price_data = caiso.get_lmp(start=start_str, end=end_str, market='DAY_AHEAD_HOURLY')

# Explore correlations between generation, demand, and price

## Notes & Findings

Use this section to document interesting patterns:
- [ ] Duck curve - when does solar peak vs demand peak?
- [ ] Weekend vs weekday patterns
- [ ] Seasonal variations
- [ ] Weather events impact
- [ ] Price spikes during low renewable generation

---

## 7. Add Cloud Cover Data from NOAA

To correlate solar generation with cloud cover, we'll use NOAA's Climate Data API.

### Setup: Get NOAA API Token
1. Visit: https://www.ncdc.noaa.gov/cdo-web/token
2. Enter your email to receive a free API token
3. Save it in a `.env` file (gitignored) or paste below

In [ ]:
# NOAA API Configuration
import requests
from typing import Optional

# Option 1: Set your token here (or use .env file)
NOAA_TOKEN = ""  # Get from: https://www.ncdc.noaa.gov/cdo-web/token

# Option 2: Load from .env file (create ../data/.env with NOAA_TOKEN=your_token)
try:
    with open('../data/.env', 'r') as f:
        for line in f:
            if line.startswith('NOAA_TOKEN='):
                NOAA_TOKEN = line.strip().split('=')[1]
except FileNotFoundError:
    pass

if not NOAA_TOKEN:
    print("⚠️  Please set NOAA_TOKEN above or create ../data/.env file")
else:
    print("✓ NOAA API token configured")

In [ ]:
# Function to fetch hourly weather data from NOAA
def fetch_noaa_weather(station_id: str, start_date: str, end_date: str, token: str) -> Optional[pd.DataFrame]:
    """
    Fetch hourly weather data including cloud cover from NOAA.
    
    Args:
        station_id: NOAA station ID (e.g., 'GHCND:USW00023174' for San Francisco)
        start_date: Start date in 'YYYY-MM-DD' format
        end_date: End date in 'YYYY-MM-DD' format
        token: NOAA API token
    
    Returns:
        DataFrame with weather observations including cloud cover
    """
    base_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
    
    headers = {'token': token}
    params = {
        'datasetid': 'NORMAL_HLY',  # Hourly data
        'stationid': station_id,
        'startdate': start_date,
        'enddate': end_date,
        'units': 'metric',
        'limit': 1000
    }
    
    cache_file = f'{DATA_DIR}/weather_{station_id.replace(":", "_")}_{start_date}_{end_date}.csv'
    
    if os.path.exists(cache_file):
        print(f"Loading weather from cache: {cache_file}")
        return pd.read_csv(cache_file, parse_dates=['date'])
    
    print(f"Fetching weather data from NOAA API...")
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        
        if 'results' in data:
            df = pd.DataFrame(data['results'])
            df.to_csv(cache_file, index=False)
            print(f"Saved to cache: {cache_file}")
            return df
        else:
            print(f"No results found. Response: {data}")
            return None
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

# Find weather stations near California solar regions
# Major stations: San Francisco, Los Angeles, Fresno, Sacramento
CALIFORNIA_STATIONS = {
    'San Francisco': 'GHCND:USW00023174',
    'Los Angeles': 'GHCND:USW00023174',
    'Fresno': 'GHCND:USW00093193',
    'Sacramento': 'GHCND:USW00023232'
}

print("California weather stations available:")
for city, station_id in CALIFORNIA_STATIONS.items():
    print(f"  {city}: {station_id}")

In [ ]:
# Fetch weather data for Sacramento (central CA, good solar proxy)
if NOAA_TOKEN:
    weather_data = fetch_noaa_weather(
        station_id=CALIFORNIA_STATIONS['Sacramento'],
        start_date=start_str,
        end_date=end_str,
        token=NOAA_TOKEN
    )
    
    if weather_data is not None:
        print(f"\nWeather data shape: {weather_data.shape}")
        print("\nAvailable data types:")
        if 'datatype' in weather_data.columns:
            print(weather_data['datatype'].unique())
        weather_data.head()
else:
    print("Set NOAA_TOKEN first to fetch weather data")

In [ ]:
# First, let's explore what datasets are available
def explore_noaa_datasets(token: str):
    """List available NOAA datasets"""
    headers = {'token': token}
    
    # Get available datasets
    print("Fetching available datasets...")
    response = requests.get('https://www.ncei.noaa.gov/cdo-web/api/v2/datasets', headers=headers)
    
    if response.status_code == 200:
        datasets = response.json()
        print(f"\nFound {len(datasets.get('results', []))} datasets:\n")
        for ds in datasets.get('results', []):
            print(f"ID: {ds['id']}")
            print(f"  Name: {ds['name']}")
            print(f"  Coverage: {ds.get('mindate', 'N/A')} to {ds.get('maxdate', 'N/A')}")
            print()
        return datasets
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

if NOAA_TOKEN:
    datasets = explore_noaa_datasets(NOAA_TOKEN)

### Alternative: Use NOAA Integrated Surface Database (ISD)

The ISD has actual hourly observations including cloud cover. Data is available via:
1. Direct download from NOAA FTP/HTTPS
2. AWS Open Data bucket

We'll use direct HTTPS access to download ISD data files.

In [ ]:
# ISD station IDs for California 
# Found by browsing: https://www.ncei.noaa.gov/data/global-hourly/access/2024/
ISD_STATIONS = {
    'Sacramento': '72483023232',  # Sacramento International Airport
}

def download_isd_data(station_id: str, year: int) -> str:
    """
    Download ISD data file for a given station and year.
    
    Args:
        station_id: ISD station ID (e.g., '72483023232')
        year: Year (e.g., 2024)
    
    Returns:
        Path to downloaded file
    """
    base_url = "https://www.ncei.noaa.gov/data/global-hourly/access"
    filename = f"{station_id}.csv"
    url = f"{base_url}/{year}/{filename}"
    
    cache_file = f"{DATA_DIR}/isd_{station_id}_{year}.csv"
    
    if os.path.exists(cache_file):
        print(f"Loading ISD data from cache: {cache_file}")
        return cache_file
    
    print(f"Downloading ISD data from: {url}")
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        # Save the CSV file
        with open(cache_file, 'wb') as f:
            f.write(response.content)
        
        print(f"Saved to: {cache_file}")
        return cache_file
        
    except Exception as e:
        print(f"Error downloading ISD data: {e}")
        print(f"Try browsing: https://www.ncei.noaa.gov/data/global-hourly/access/{year}/")
        return None

# Download 2024 data for Sacramento
print("Downloading 2024 weather data for Sacramento...")
test_file = download_isd_data(ISD_STATIONS['Sacramento'], 2024)

if test_file:
    # Load and preview the data
    isd_data = pd.read_csv(test_file, low_memory=False)
    print(f"\n✓ Successfully loaded ISD data!")
    print(f"Data shape: {isd_data.shape}")
    print(f"\nColumns available ({len(isd_data.columns)} total):")
    for i, col in enumerate(isd_data.columns):
        print(f"  {i+1}. {col}")
    
    print("\nFirst few rows:")
    print(isd_data.head())
    
    # Check for cloud cover columns
    cloud_cols = [col for col in isd_data.columns if 'cloud' in col.lower() or 'sky' in col.lower()]
    if cloud_cols:
        print(f"\n☁️  Cloud-related columns found: {cloud_cols}")
else:
    print("\n❌ Failed to download.")

In [ ]:
# Download NOAA ISD station history file
station_history_url = "https://www.ncei.noaa.gov/pub/data/noaa/isd-history.txt"
station_cache = f"{DATA_DIR}/isd_station_history.txt"

if not os.path.exists(station_cache):
    print("Downloading ISD station history...")
    response = requests.get(station_history_url)
    with open(station_cache, 'wb') as f:
        f.write(response.content)
    print(f"Saved to: {station_cache}")
else:
    print(f"Loading from cache: {station_cache}")

# Parse the fixed-width station history file
# Column positions based on ISD documentation
colspecs = [
    (0, 6),      # USAF
    (7, 12),     # WBAN
    (13, 42),    # STATION NAME
    (43, 45),    # CTRY (Country)
    (48, 50),    # ST (State)
    (51, 56),    # ICAO
    (57, 64),    # LAT (latitude * 1000)
    (65, 73),    # LON (longitude * 1000)
    (74, 81),    # ELEV (elevation in meters * 10)
    (82, 90),    # BEGIN
    (91, 99),    # END
]

names = ['USAF', 'WBAN', 'STATION_NAME', 'CTRY', 'ST', 'ICAO', 
         'LAT', 'LON', 'ELEV', 'BEGIN', 'END']

# Read fixed-width format, skip header
stations = pd.read_fwf(station_cache, colspecs=colspecs, names=names, skiprows=20)

# Filter for California stations
ca_stations = stations[stations['ST'] == 'CA'].copy()

# Convert coordinates - handle missing values marked with '+'
ca_stations['LAT'] = pd.to_numeric(ca_stations['LAT'], errors='coerce') / 1000
ca_stations['LON'] = pd.to_numeric(ca_stations['LON'], errors='coerce') / 1000
ca_stations['ELEV'] = pd.to_numeric(ca_stations['ELEV'], errors='coerce') / 10

# Remove stations with missing coordinates
ca_stations = ca_stations.dropna(subset=['LAT', 'LON'])

# Create station ID (USAF + WBAN)
ca_stations['STATION_ID'] = ca_stations['USAF'].astype(str).str.zfill(6) + ca_stations['WBAN'].astype(str).str.zfill(5)

# Filter for stations with recent data (active in 2024)
ca_stations['END'] = pd.to_datetime(ca_stations['END'], format='%Y%m%d', errors='coerce')
recent_stations = ca_stations[ca_stations['END'] >= '2024-01-01'].copy()

print(f"\nFound {len(ca_stations)} California stations with valid coordinates")
print(f"Found {len(recent_stations)} stations with data through 2024+")
print(f"\nCalifornia stations (sorted by latitude, north to south):")
recent_stations_sorted = recent_stations.sort_values('LAT', ascending=False)
print(recent_stations_sorted[['STATION_ID', 'STATION_NAME', 'LAT', 'LON', 'ELEV']].head(30).to_string())

In [ ]:
# Download NOAA ISD station history file
station_history_url = "https://www.ncei.noaa.gov/pub/data/noaa/isd-history.txt"
station_cache = f"{DATA_DIR}/isd_station_history.txt"

if not os.path.exists(station_cache):
    print("Downloading ISD station history...")
    response = requests.get(station_history_url)
    with open(station_cache, 'wb') as f:
        f.write(response.content)
    print(f"Saved to: {station_cache}")
else:
    print(f"Loading from cache: {station_cache}")

# Parse the fixed-width station history file
# Column positions based on ISD documentation
colspecs = [
    (0, 6),      # USAF
    (7, 12),     # WBAN
    (13, 42),    # STATION NAME
    (43, 45),    # CTRY (Country)
    (48, 50),    # ST (State)
    (51, 56),    # ICAO
    (57, 64),    # LAT (latitude * 1000)
    (65, 73),    # LON (longitude * 1000)
    (74, 81),    # ELEV (elevation in meters * 10)
    (82, 90),    # BEGIN
    (91, 99),    # END
]

names = ['USAF', 'WBAN', 'STATION_NAME', 'CTRY', 'ST', 'ICAO', 
         'LAT', 'LON', 'ELEV', 'BEGIN', 'END']

# Read fixed-width format, skip header
stations = pd.read_fwf(station_cache, colspecs=colspecs, names=names, skiprows=20)

# Filter for California stations
ca_stations = stations[stations['ST'] == 'CA'].copy()

# Convert coordinates (stored as integers * 1000) to decimal degrees
ca_stations['LAT'] = ca_stations['LAT'] / 1000
ca_stations['LON'] = ca_stations['LON'] / 1000
ca_stations['ELEV'] = ca_stations['ELEV'] / 10

# Create station ID (USAF + WBAN)
ca_stations['STATION_ID'] = ca_stations['USAF'].astype(str).str.zfill(6) + ca_stations['WBAN'].astype(str).str.zfill(5)

# Filter for stations with recent data (active in 2024)
ca_stations['END'] = pd.to_datetime(ca_stations['END'], format='%Y%m%d', errors='coerce')
recent_stations = ca_stations[ca_stations['END'] >= '2024-01-01']

print(f"\nFound {len(ca_stations)} total California stations")
print(f"Found {len(recent_stations)} stations with data through 2024+")
print(f"\nTop California stations (by name):")
print(recent_stations[['STATION_ID', 'STATION_NAME', 'LAT', 'LON', 'ELEV']].head(20))

## 8. Find California Weather Stations

Let's download the NOAA ISD station inventory to find all California stations with cloud cover data.

In [ ]:
# Identify cloud-related columns
# Based on ISD documentation, cloud columns typically include:
# - CIG (ceiling height)
# - Columns starting with 'AA', 'GA', 'GD', 'GF' (sky cover/cloud layers)

print("Searching for cloud-related columns...")
print("\nColumns containing 'CIG' (ceiling):")
cig_cols = [col for col in isd_data.columns if 'CIG' in col.upper()]
print(cig_cols if cig_cols else "None found")

print("\nColumns starting with cloud layer codes (AA, GA, GD, GF):")
cloud_layer_cols = [col for col in isd_data.columns if col.startswith(('AA', 'GA', 'GD', 'GF'))]
print(cloud_layer_cols if cloud_layer_cols else "None found")

print("\nAll columns (to manually inspect):")
for i, col in enumerate(isd_data.columns, 1):
    non_null = isd_data[col].notna().sum()
    if non_null > 0:  # Only show columns with some data
        print(f"{i:3d}. {col:40s} - {non_null:,} non-null values ({non_null/len(isd_data)*100:.1f}%)")

# Show non-null percentage for potential cloud columns
print("\n" + "="*70)
print("Looking for columns with data that might be cloud-related...")
print("="*70)

In [ ]:
# Explore the main cloud cover columns
print("Cloud Cover Column Details:")
print("="*70)

# Parse the cloud data - ISD format is comma-delimited within each field
# GF1 = Sky condition observation (total coverage)
# GA1/2/3 = Individual cloud layers
# CIG = Ceiling height

print("\n1. CIG (Ceiling) - Sample values:")
print(isd_data['CIG'].head(20))

print("\n2. GF1 (Sky Condition) - Sample values:")
print(isd_data['GF1'].dropna().head(20))

print("\n3. GA1 (Cloud Layer 1) - Sample values:")
print(isd_data['GA1'].dropna().head(20))

print("\n4. GD1 (Sky Cover Summation) - Sample values:")
print(isd_data['GD1'].dropna().head(20))

# Parse DATE column to datetime for time series analysis
print("\n5. Date range in this data:")
isd_data['datetime'] = pd.to_datetime(isd_data['DATE'])
print(f"   Start: {isd_data['datetime'].min()}")
print(f"   End: {isd_data['datetime'].max()}")
print(f"   Total observations: {len(isd_data):,}")

In [ ]:
# Parse cloud cover from GD1 (Sky Cover Summation)
# Format: "oktas,qualifier,quality,height,quality,quality"
# We want the first value (oktas: 0-8 scale, 9=missing)

def parse_cloud_cover(gd1_value):
    """Extract cloud cover in oktas from GD1 field"""
    if pd.isna(gd1_value):
        return None
    try:
        oktas = int(str(gd1_value).split(',')[0])
        return oktas if oktas != 9 else None  # 9 = missing data
    except:
        return None

# Parse cloud cover
isd_data['cloud_cover_oktas'] = isd_data['GD1'].apply(parse_cloud_cover)

# Convert oktas to percentage (oktas/8 * 100)
isd_data['cloud_cover_pct'] = isd_data['cloud_cover_oktas'] / 8 * 100

# Show distribution
print("Cloud Cover Distribution (2024 Sacramento):")
print("="*70)
print(isd_data['cloud_cover_oktas'].value_counts().sort_index())

print("\nCloud cover statistics:")
print(f"  Mean: {isd_data['cloud_cover_oktas'].mean():.1f} oktas ({isd_data['cloud_cover_pct'].mean():.1f}%)")
print(f"  Median: {isd_data['cloud_cover_oktas'].median():.1f} oktas")
print(f"  Clear sky (0 oktas): {(isd_data['cloud_cover_oktas']==0).sum():,} observations")
print(f"  Overcast (7-8 oktas): {(isd_data['cloud_cover_oktas']>=7).sum():,} observations")

# Preview the parsed data
print("\nSample of parsed cloud data:")
print(isd_data[['datetime', 'cloud_cover_oktas', 'cloud_cover_pct']].head(20))